In [0]:
raw_df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("/Volumes/workspace/default/data_file/data/Online_Retail.csv")

raw_df.write.format("delta").mode("overwrite").save("/mnt/bronze/sales_data")

In [0]:
bronze_df = spark.read.format("delta").load("/mnt/bronze/sales_data")

silver_df = bronze_df.filter("Quantity > 0 AND UnitPrice > 0") \
    .withColumnRenamed("InvoiceNo", "invoice_no") \
    .withColumnRenamed("StockCode", "stock_code")

silver_df.write.format("delta").mode("overwrite").save("/mnt/silver/sales_data_cleaned")

In [0]:
silver_df = spark.read.format("delta").load("/mnt/silver/sales_data_cleaned")

gold_df = silver_df.groupBy("Country") \
    .agg({"Quantity": "sum", "UnitPrice": "avg"}) \
    .withColumnRenamed("sum(Quantity)", "total_quantity") \
    .withColumnRenamed("avg(UnitPrice)", "avg_unit_price")

gold_df.write.format("delta").mode("overwrite").save("/mnt/gold/sales_summary")